# Analysis of Capital, Fixed, and Variable Costs for Various Generation Types

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Many of these reports are from previous years, so we should account for this by adjusting for inflation. 
We can do this with the ``cpi`` (consumer price index) from the Bureau of Labor Statistics.

$$ P_{adj} = \frac{P_{old} \cdot CPI_{present}}{CPI_{old}} $$

We can retreive the consumer price index data from the Bureau of Labor Statistics. The usual values for inflation calculations are the "All Urban Consumers (CU)" survey. The code for this is "CUUR0000SA0". 

In [2]:
url = "https://download.bls.gov/pub/time.series/cu/cu.data.0.Current"
cpi_data = pd.read_csv(url, sep='\t')

In [3]:
cpi_key = cpi_data.keys()[0]
cpi_key

'series_id        '

In [4]:
cu = list(cpi_data[cpi_key])[0]
cu

'CUSR0000SA0      '

In [5]:
cpi_u = cpi_data[cpi_key] == cu
cpi_data = cpi_data[cpi_u]

In [6]:
cpi_data.index = pd.to_datetime(cpi_data.iloc[:,1], format='%Y')
cpi_data

,series_id,year,period,value,footnote_codes
year,,,,,
1997-01-01,CUSR0000SA0,1997,M01,159.400,NaN
1997-01-01,CUSR0000SA0,1997,M02,159.700,NaN
1997-01-01,CUSR0000SA0,1997,M03,159.800,NaN
1997-01-01,CUSR0000SA0,1997,M04,159.900,NaN
1997-01-01,CUSR0000SA0,1997,M05,159.900,NaN
...,...,...,...,...,...
2020-01-01,CUSR0000SA0,2020,M11,260.927,NaN
2020-01-01,CUSR0000SA0,2020,M12,261.560,NaN
2021-01-01,CUSR0000SA0,2021,M01,262.231,NaN


In [7]:
cpi_average = cpi_data.resample('Y').mean()
cpi_average

,year,value,footnote_codes
year,,,
1997-12-31,1997,160.525000,NaN
1998-12-31,1998,163.008333,NaN
1999-12-31,1999,166.583333,NaN
2000-12-31,2000,172.191667,NaN
2001-12-31,2001,177.041667,NaN
2002-12-31,2002,179.866667,NaN
2003-12-31,2003,184.000000,NaN
2004-12-31,2004,188.908333,NaN
2005-12-31,2005,195.266667,NaN


In [8]:
target_year = 2021
cpi_current = cpi_average[cpi_average.iloc[:,0]==target_year].iloc[:,1]

In [9]:
def inflate(p_old, start_year, target_year=2021):
    """
    This function calculates the inflated value of an item
    based on the Consumer Price Index from the Bureau of 
    Labor Statistics.
    
    Parameters
    ----------
    p_old : float
        The old price or value of an item. I.e. the price of
        the item at the starting year.
    start_year : integer
        The year from which you would like to inflate. E.g.
        the price of milk in 1997 inflated to today's dollars.
    target_year : integer
        The target year you would like to inflate to. E.g.
        the price of milk in 1997 inflated to 2021 dollars.
        Default is 2021.
        
    Returns
    -------
    p_adj : float
        The price adjusted for inflation.
    """
    
    cpi_current = float(cpi_average[cpi_average.iloc[:,0]==target_year].iloc[:,1])    
    cpi_old = float(cpi_average[cpi_average.iloc[:,0]==start_year].iloc[:,1])
    
    p_adj = (p_old*cpi_current)/cpi_old
    
    return p_adj

In [10]:
inflate(1.00, 2017, 2017)

1.0

In [11]:
inflate(1.00, 2017, 2020)

1.0559224728092558

In [12]:
inflate(1.00, 2017, 1997)

0.6548419139618701

## Nuclear Plants

"Nuclear Costs in Context" (NEI, 2018)

This document gives a cost summary for each of fuel, capital, and operating costs in \$/MWh. This is 
helpful for variable operating costs. But we cannot reliably convert these into fixed values required for capital and fixed annual costs. 

In [13]:
fuel_nei_bwr = inflate(6.22, 2017)/1e6  # M$/MWh
fuel_nei_pwr = inflate(6.55, 2017)/1e6  # M$/MWh
print(fuel_nei_bwr);print(fuel_nei_pwr)

6.683311474674212e-06
7.037892308539563e-06


In [14]:
om_nei_bwr_total = inflate(27.81, 2017)/1e6  # M$/MWh
om_nei_pwr_total = inflate(26.36, 2017)/1e6  # M$/MWh
print(om_nei_bwr_total);print(om_nei_pwr_total)

2.988149390847103e-05
2.8323487214214184e-05


"Projected Costs of Generating Electricity" (IEA, 2020)

This document only gives a cost summary for overnight capital costs in \$/kWe. This is helpful for investment costs but gives no indication for annual fixed costs. It also does not distinguish between PWR and BWR for the United States.

It DOES give values for capital costs required for license renewals (if we were to incorporate this, I believe we should implement as an additional annual fixed cost, dividing by the life of the extension). 

In [15]:
# New Build for Nuclear
cc_iea = inflate(4250, 2020)*1000/1e6  # M$/MW
print(cc_iea)

4.324722186361225


"Nuclear Power Economics and Project Structuring" (WNA, 2017)

This document gives OM and Fuel Costs (cents per kWh) as well as estimated capital cost -- assumes PWR

In [16]:
om_total_wna = inflate(0.024, 2011)*1000/1e6  # M$/MWh
cc_wna_pwr = inflate(4100, 2015)*1000/1e6  # M$/MW
print(om_total_wna);print(cc_wna_pwr)

2.8105084851260213e-05
4.556588717171921


"Capital Costs and Performance Characteristics for Utility Scale Power Generating Technologies" (Sargent and Lundy, 2020)


This document gives capital cost, dollars/kW, variable OM, dollars/MWh, and fixed OM, \$ /kW-year. However, not for current technology in the U.S. Estimates for "advanced" (AP-1000) and "modular" (small modular reactors, 50 MW).

Also provided: Specific emissions information (NOx, SOx, CO2)

In [17]:
cc_sl_adv = inflate(6041, 2020)*1000/1e6  # M$/MW
cc_sl_smr = inflate(6191, 2020)*1000/1e6  # M$/MW
print(cc_sl_adv);print(cc_sl_smr)
om_sl_adv_fix = inflate(121.64, 2020)*1000/1e6  # M$/MW-year
om_sl_smr_fix = inflate(95.00, 2020)*1000/1e6  # M$/MW-year
print(om_sl_adv_fix);print(om_sl_smr_fix)
om_sl_adv_var = inflate(2.37, 2020)/1e6  # M$/MWh
om_sl_smr_var = inflate(3.00, 2020)/1e6  # M$/MWh
print(om_sl_adv_var);print(om_sl_smr_var)

6.147210994778389
6.299848248414667
0.12377863688211277
0.0966702606363097
2.4116686074532e-06
3.0527450727255697e-06


Based on the values from the Sargent and Lundy report, we can make a reasonable guess about the 
annual fixed cost of a BWR or PWR from the NEI-2018 report.

Assumptions:

1. The "capital" cost in \$/MWh corresponds to the annual "fixed" cost.
2. The "total operating" cost in \$/MWh corresponds to the "variable" cost.
3. The average capacity factor is 92\%. This will be used to convert MWh to MW

In [18]:
mwh_to_mw = 8760*0.92
om_nei_bwr_fix = inflate(6.63*mwh_to_mw, 2017)/1e6  # M$/MW-year
om_nei_pwr_fix = inflate(6.64*mwh_to_mw, 2017)/1e6  # M$/MW-year
print(om_nei_bwr_fix);print(om_nei_pwr_fix)

0.05741254238541542
0.057499137471969594


"Capital Cost Estimates for Utility Scale Electricity Generating Plants" (EIA, 2016)

This document gives
* Overnight capital cost \$/kW
* Fixed OM cost \$/kW-year
* Variable OM cost \$/MWh

for "advanced nuclear," presumably AP-1000

In [19]:
cc_eia = inflate(5945, 2016)*1000/1e6  # M$/MW
print(cc_eia)
om_eia_fix = inflate(100.28, 2016)*1000/1e6  # M$/MW-year
print(om_eia_fix)
om_eia_var = inflate(2.3, 2016)/1e6  # M$/MWh
print(om_eia_var)

6.524366394508458
0.11005272700442527
2.524145114780396e-06


In [20]:
nuclear_pd = {'capital':[np.nan,np.nan,
                         cc_iea,
                         cc_wna_pwr,
                         cc_sl_adv, cc_sl_smr,
                         cc_eia],  # M$/MW
              'fixed':[om_nei_pwr_fix,om_nei_bwr_fix,
                       np.nan,
                       np.nan,
                       om_sl_adv_fix, om_sl_smr_fix,
                       om_eia_fix],  # M$/MW-year
              'variable':[om_nei_pwr_total, om_nei_bwr_total,
                          np.nan,
                          om_total_wna,
                          om_sl_adv_var, om_sl_smr_var,
                          om_eia_var],  # M$/MWh
              'type':['PWR', 'BWR',
                      'LWR',
                      'PWR',
                      'advanced', 'modular',
                      'advanced'],  # BWR, PWR, advanced, modular
              'source':['NEI-2018','NEI-2018',
                        'IEA-2020',
                        'WNA-2017',
                        'SL-2020', 'SL-2020',
                        'EIA-2016'],
              'notes':['calculated from assumptions tabulated in cost_analysis.ipynb',
                       'calculated from assumptions tabulated in cost_analysis.ipynb',
                       'source does not give annual fixed or variable costs',
                       'source does not give annual fixed cost',
                       'source specifies \"advanced\" as AP-1000',
                       'source specifies \"modular\" as 12x50-MW capacity',
                       'source only gives values for \"advanced\" nuclear']
             }  

nuclear_df = pd.DataFrame(nuclear_pd)
nuclear_df

,capital,fixed,variable,type,source,notes
0,NaN,0.057499,0.000028,PWR,NEI-2018,calculated from assumptions tabulated in cost_...
1,NaN,0.057413,0.000030,BWR,NEI-2018,calculated from assumptions tabulated in cost_...
2,4.324722,NaN,NaN,LWR,IEA-2020,source does not give annual fixed or variable ...
3,4.556589,NaN,0.000028,PWR,WNA-2017,source does not give annual fixed cost
4,6.147211,0.123779,0.000002,advanced,SL-2020,"source specifies ""advanced"" as AP-1000"
5,6.299848,0.096670,0.000003,modular,SL-2020,"source specifies ""modular"" as 12x50-MW capacity"
6,6.524366,0.110053,0.000003,advanced,EIA-2016,"source only gives values for ""advanced"" nuclear"


## Natural Gas Plants

Sources: (EIA, 2016) same as above, (Sargent and Lundy, 2020) same as above

For new natural gas builds, I assume "Natural Gas Combined Cycle."

In [21]:
cc_sl_ngcc = inflate(958, 2020)*1000/1e6  # M$/MW
om_sl_fix_ngcc = inflate(12.20, 2020)*1000/1e6  # M$/MW-year
om_sl_var_ngcc = inflate(1.87, 2020)/1e6 # M$/MWh

cc_eia_ngcc = inflate(978, 2016)*1000/1e6  # M$/MW
om_eia_fix_ngcc = inflate(11, 2016)*1000/1e6  # M$/MW-year
om_eia_var_ngcc = inflate(3.5, 2016)/1e6  # M$/MWh

print(f"According to Sargent and Lundy: NGCC")
print(f"Capital Cost: {cc_sl_ngcc}; Fixed Cost: {om_sl_fix_ngcc}; Variable Cost: {om_sl_var_ngcc}")
print(f"According to EIA: NGCC")
print(f"Capital Cost: {cc_eia_ngcc}; Fixed Cost: {om_eia_fix_ngcc}; Variable Cost: {om_eia_var_ngcc}")

According to Sargent and Lundy: NGCC
Capital Cost: 0.9748432598903651; Fixed Cost: 0.012414496629083982; Variable Cost: 1.9028777619989385e-06
According to EIA: NGCC
Capital Cost: 1.0733104009805337; Fixed Cost: 0.012071998375036676; Variable Cost: 3.841090392057124e-06


## Onshore Wind Farms

Sources: (EIA, 2016) same as above, (Sargent and Lundy, 2020) same as above, (IEA, 2020) same as above

In [22]:
cc_eia_onw = inflate(1877, 2016)*1000/1e6
om_eia_fix_onw = inflate(39.7, 2016)*1000/1e6

cc_sl_onw = inflate(1265, 2020)*1000/1e6
om_sl_fix_onw = inflate(26.34, 2020)*1000/1e6

cc_iea_onw = inflate(1968, 2020)*1000/1e6

print(f"According to Sargent and Lundy: Onshore Wind")
print(f"Capital Cost: {cc_sl_onw}; Fixed Cost: {om_sl_fix_onw}; Variable Cost: {0}")
print(f"According to EIA: Onshore Wind")
print(f"Capital Cost: {cc_eia_onw}; Fixed Cost: {om_eia_fix_onw}; Variable Cost: {0}")
print(f"According to IEA: Onshore Wind")
print(f"Capital Cost: {cc_iea_onw}; Fixed Cost: {np.nan}; Variable Cost: {0}")

According to Sargent and Lundy: Onshore Wind
Capital Cost: 1.2872408389992818; Fixed Cost: 0.026803101738530503; Variable Cost: 0
According to EIA: Onshore Wind
Capital Cost: 2.0599219045403494; Fixed Cost: 0.043568939589905105; Variable Cost: 0
According to IEA: Onshore Wind
Capital Cost: 2.002600767707974; Fixed Cost: nan; Variable Cost: 0


Since the IEA and S\&L reports are equally recent, we will use an intermediate value for the capital cost.

In [23]:
print(f"Mean capital cost for onshore windfarms {(cc_iea_onw+cc_sl_onw)/2}")

Mean capital cost for onshore windfarms 1.6449208033536278


## Solar Farms

Sources: (EIA, 2016) same as above, (Sargent and Lundy, 2020) same as above, (IEA, 2020) same as above

Assume single axis tracking

In [24]:
cc_eia_sol = inflate(2534, 2016)*1000/1e6
om_eia_fix_sol = inflate(21.8, 2016)*1000/1e6

cc_sl_sol = inflate(1313, 2020)*1000/1e6
om_sl_fix_sol = inflate(15.25, 2020)*1000/1e6

cc_iea_sol = inflate(1072, 2020)*1000/1e6

print(f"According to Sargent and Lundy: Solar Farm (Utility Scale)")
print(f"Capital Cost: {cc_sl_sol}; Fixed Cost: {om_sl_fix_sol}; Variable Cost: {0}")
print(f"According to EIA: Solar Farm (Utility Scale)")
print(f"Capital Cost: {cc_eia_sol}; Fixed Cost: {om_eia_fix_sol}; Variable Cost: {0}")
print(f"According to IEA: Solar Farm (Utility Scale)")
print(f"Capital Cost: {cc_iea_sol}; Fixed Cost: {np.nan}; Variable Cost: {0}")

According to Sargent and Lundy: Solar Farm (Utility Scale)
Capital Cost: 1.3360847601628911; Fixed Cost: 0.01551812078635498; Variable Cost: 0
According to EIA: Solar Farm (Utility Scale)
Capital Cost: 2.780949443849358; Fixed Cost: 0.023924505870527236; Variable Cost: 0
According to IEA: Solar Farm (Utility Scale)
Capital Cost: 1.090847572653937; Fixed Cost: nan; Variable Cost: 0


Since IEA doesn't specify if it's fixed or single axis tracking we will use the Sargent and Lundy values.

## Coal Plants

Data about the annual fixed costs for conventional coal plants (i.e. not supercritical, no carbon capture) is difficult to find. Fortunately, most coal plants were built before 1990 and thus they are likely "fully depreciated" in 2021. Therefore we can use the LCOE estimate from [Lazard](https://www.lazard.com/perspective/levelized-cost-of-energy-and-levelized-cost-of-storage-2020/).

Estimate for fully depreciated: \$41/MWh


#### New Coal
For new coal plants, we assume ultra-supercritical with 90\% carbon capture storage.

Sources: (EIA, 2016) same as above, (Sargent and Lundy, 2020) same as above, (IEA, 2020) same as above

In [25]:
lcoe_coal_laz = inflate(41, 2020)/1e6
print(lcoe_coal_laz)

4.1720849327249454e-05


In [26]:
cc_eia_coal = inflate(5084, 2016)*1000/1e6
om_eia_fix_coal = inflate(70, 2016)*1000/1e6
om_eia_var_coal = inflate(7.1, 2016)*1000/1e6

cc_sl_coal = inflate(5876, 2020)*1000/1e6
om_sl_fix_coal = inflate(59.54, 2020)*1000/1e6
om_sl_var_coal = inflate(10.98, 2020)/1e6

cc_iea_coal = inflate(5991, 2020)*1000/1e6

print(f"According to Sargent and Lundy: Coal (UCS/CCS)")
print(f"Capital Cost: {cc_sl_coal}; Fixed Cost: {om_sl_fix_coal}; Variable Cost: {om_sl_var_coal}")
print(f"According to EIA: Solar Farm (Utility Scale)")
print(f"Capital Cost: {cc_eia_coal}; Fixed Cost: {om_eia_fix_coal}; Variable Cost: {om_eia_var_coal}")
print(f"According to IEA: Solar Farm (Utility Scale)")
print(f"Capital Cost: {cc_iea_coal}; Fixed Cost: {np.nan}; Variable Cost: {np.nan}")

According to Sargent and Lundy: Coal (UCS/CCS)
Capital Cost: 5.979310015778483; Fixed Cost: 0.06058681387669347; Variable Cost: 1.1173046966175586e-05
According to EIA: Solar Farm (Utility Scale)
Capital Cost: 5.579458158062405; Fixed Cost: 0.0768218078411425; Variable Cost: 0.007791926223887309
According to IEA: Solar Farm (Utility Scale)
Capital Cost: 6.096331910232963; Fixed Cost: nan; Variable Cost: nan


### Update: NREL Annual Technology Baseline

In [27]:
coal_existing_capital = inflate(3603*1000/1e6, 2020)
print(f"Investment cost for existing coal {coal_existing_capital} (M$/MW)")
coal_new_capital = inflate(5931*1000/1e6, 2020)
print(f"Investment cost for new coal {coal_new_capital} (M$/MW)")
coal_existing_fixed = inflate(40*1000/1e6, 2020)
print(f"Annual fixed cost for existing coal {coal_existing_fixed} (M$/MW-year)")
coal_new_fixed = inflate(58*1000/1e6, 2020)
print(f"Annual fixed cost for new coal {coal_new_fixed} (M$/MW-year)")
coal_existing_variable = inflate((4+17)/1e6, 2020)
print(f"Variable cost for existing coal {coal_existing_variable} (M$/MWh)")
coal_new_variable = inflate((11+25)/1e6, 2020)
print(f"Variable cost for new coal {coal_new_variable} (M$/MWh)")

Investment cost for existing coal 3.666346832343409 (M$/MW)
Investment cost for new coal 6.0352770087784515 (M$/MW)
Annual fixed cost for existing coal 0.04070326763634093 (M$/MW-year)
Annual fixed cost for new coal 0.059019738072694346 (M$/MW-year)
Variable cost for existing coal 2.1369215509078985e-05 (M$/MWh)
Variable cost for new coal 3.663294087270684e-05 (M$/MWh)


In [28]:
natgas_existing_capital = inflate(943*1000/1e6, 2020)
print(f"Investment cost for existing natgas {natgas_existing_capital} (M$/MW)")
natgas_new_capital = inflate(2666*1000/1e6, 2020)
print(f"Investment cost for new natgas {natgas_new_capital} (M$/MW)")
natgas_existing_fixed = inflate(11*1000/1e6, 2020)
print(f"Annual fixed cost for existing natgas {natgas_existing_fixed} (M$/MW-year)")
natgas_new_fixed = inflate(27*1000/1e6, 2020)
print(f"Annual fixed cost for new natgas {natgas_new_fixed} (M$/MW-year)")
natgas_existing_variable = inflate((4+18)/1e6, 2020)
print(f"Variable cost for existing natgas {natgas_existing_variable} (M$/MWh)")
natgas_new_variable = inflate((6+21)/1e6, 2020)
print(f"Variable cost for new natgas {natgas_new_variable} (M$/MWh)")

Investment cost for existing natgas 0.9595795345267374 (M$/MW)
Investment cost for new natgas 2.7128727879621226 (M$/MW)
Annual fixed cost for existing natgas 0.011193398599993755 (M$/MW-year)
Annual fixed cost for new natgas 0.027474705654530127 (M$/MW-year)
Variable cost for existing natgas 2.238679719998751e-05 (M$/MWh)
Variable cost for new natgas 2.7474705654530124e-05 (M$/MWh)


In [29]:
nuclear_existing_capital = inflate(943*1000/1e6, 2020)
print(f"Investment cost for existing nuclear {nuclear_existing_capital} (M$/MW)")
nuclear_new_capital = inflate(6125*1000/1e6, 2020)
print(f"Investment cost for new nuclear {nuclear_new_capital} (M$/MW)")
nuclear_existing_fixed = inflate(11*1000/1e6, 2020)
print(f"Annual fixed cost for existing nuclear {nuclear_existing_fixed} (M$/MW-year)")
nuclear_new_fixed = inflate(119*1000/1e6, 2020)
print(f"Annual fixed cost for new nuclear {nuclear_new_fixed} (M$/MW-year)")
nuclear_existing_variable = inflate((4+3)/1e6, 2020)
print(f"Variable cost for existing nuclear {nuclear_existing_variable} (M$/MWh)")
nuclear_new_variable = inflate((7+2)/1e6, 2020)
print(f"Variable cost for new nuclear {nuclear_new_variable} (M$/MWh)")

Investment cost for existing nuclear 0.9595795345267374 (M$/MW)
Investment cost for new nuclear 6.232687856814705 (M$/MW)
Annual fixed cost for existing nuclear 0.011193398599993755 (M$/MW-year)
Annual fixed cost for new nuclear 0.12109222121811426 (M$/MW-year)
Variable cost for existing nuclear 7.123071836359662e-06 (M$/MWh)
Variable cost for new nuclear 9.15823521817671e-06 (M$/MWh)


In [30]:
utilitypv_new_capital = inflate(1566*1000/1e6, 2020)
print(f"Investment cost for new utility pv {utilitypv_new_capital} (M$/MW)")
utilitypv_new_fixed = inflate(19*1000/1e6, 2020)
print(f"Annual fixed cost for new utility pv {utilitypv_new_fixed} (M$/MW-year)")
utilitypv_new_variable = inflate((0)/1e6, 2020)
print(f"Variable cost for new utility pv {utilitypv_new_variable} (M$/MWh)")

Investment cost for new utility pv 1.5935329279627475 (M$/MW)
Annual fixed cost for new utility pv 0.01933405212726194 (M$/MW-year)
Variable cost for new utility pv 0.0 (M$/MWh)


In [31]:
residentialpv_new_capital = inflate(3054*1000/1e6, 2020)
print(f"Investment cost for new residential pv {residentialpv_new_capital} (M$/MW)")
residentialpv_new_fixed = inflate(22*1000/1e6, 2020)
print(f"Annual fixed cost for new residential pv {residentialpv_new_fixed} (M$/MW-year)")
residentialpv_new_variable = inflate((0)/1e6, 2020)
print(f"Variable cost for new residential pv {residentialpv_new_variable} (M$/MWh)")

Investment cost for new residential pv 3.1076944840346297 (M$/MW)
Annual fixed cost for new residential pv 0.02238679719998751 (M$/MW-year)
Variable cost for new residential pv 0.0 (M$/MWh)


In [32]:
wind_new_capital = inflate(1846*1000/1e6, 2020)
print(f"Investment cost for new wind pv {wind_new_capital} (M$/MW)")
wind_new_fixed = inflate(43*1000/1e6, 2020)
print(f"Annual fixed cost for new wind pv {wind_new_fixed} (M$/MW-year)")
wind_new_variable = inflate((0)/1e6, 2020)
print(f"Variable cost for new wind pv {wind_new_variable} (M$/MWh)")

Investment cost for new wind pv 1.878455801417134 (M$/MW)
Annual fixed cost for new wind pv 0.0437560127090665 (M$/MW-year)
Variable cost for new wind pv 0.0 (M$/MWh)


In [33]:
def convert_to_MTperMWh(emission):
    '''
    Converts a number in units of lb/MMBtu to MT/MWh
    
    Inputs: 
    -------
    emission: float
        reported value in units of lb/MMBtu
    
    Outputs:
    --------
    converted_emission: float
        converted reported value in units of MT/MWh
    '''
    mmbtu_to_mwh = emission/0.29307107
    lb_to_kg = mmbtu_to_mwh*0.453592
    kg_to_tonne = lb_to_kg/1000
    tonne_to_MT = kg_to_tonne/1e6
    converted_emission = tonne_to_MT
#     converted_emission = emission*1.54772e-3
    return converted_emission

In [34]:
to_MTperMWh = lambda g: g/1e9  # MT / MWh

In [35]:
cost_df = {'technology':['coal_existing',
                         'coal_new',
                         'natgas_existing',
                         'natgas_new',
                         'nuclear_existing',
                         'nuclear_new',
                         'solar_utility',
                         'solar_residential',
                         'wind_utility',
                         'li_battery'],
           'investment (M$/MW)':[coal_existing_capital,
                                 coal_new_capital,
                                 natgas_existing_capital,
                                 natgas_new_capital,
                                 391*1000/1e6,
                                 nuclear_new_capital,
                                 utilitypv_new_capital,
                                 residentialpv_new_capital,
                                 wind_new_capital,
                                 0.0],
           'fixed (M$/MW-year)':[coal_existing_fixed,
                                 coal_new_fixed,
                                 natgas_existing_fixed,
                                 natgas_new_fixed,
                                 om_nei_bwr_fix,
                                 nuclear_new_fixed,
                                 utilitypv_new_fixed,
                                 residentialpv_new_fixed,
                                 wind_new_fixed,
                                 0.0],
           'variable (M$/MWh)':[coal_existing_variable,
                                coal_new_variable,
                                natgas_existing_variable,
                                natgas_new_variable,
                                om_nei_bwr_total,
                                nuclear_new_variable,
                                utilitypv_new_variable,
                                residentialpv_new_variable,
                                wind_new_variable,
                                0.0],
           'SO2 (MT/MWh)':[convert_to_MTperMWh(0.1000),
                           convert_to_MTperMWh(0.0555),
                           convert_to_MTperMWh(0.0033),
                           convert_to_MTperMWh(0.0033),
                           0.0,
                           0.0,
                           0.0,
                           0.0,
                           0.0,
                           0.0],
           'NOx (MT/MWh)':[convert_to_MTperMWh(0.080),
                           convert_to_MTperMWh(0.085),
                           convert_to_MTperMWh(0.02),
                           convert_to_MTperMWh(0.02),
                           0.0,
                           0.0,
                           0.0,
                           0.0,
                           0.0,
                           0.0],
           'CO2 (MT/MWh)':[convert_to_MTperMWh(210.6),
                           convert_to_MTperMWh(21.1),
                           convert_to_MTperMWh(117.0),
                           convert_to_MTperMWh(11.7),
                           0.0,
                           0.0,
                           0.0,
                           0.0,
                           0.0,
                           0.0],
           'Hg (MT/MWh)':[convert_to_MTperMWh(4.361e-6),
                          convert_to_MTperMWh(4.361e-7),
                          0.0,
                          0.0,
                          0.0,
                          0.0,
                          0.0,
                          0.0,
                          0.0,
                          0.0],
           'CO2eq (MT/MWh)':[to_MTperMWh(820),
                             to_MTperMWh(220),
                             to_MTperMWh(490),
                             to_MTperMWh(170),
                             to_MTperMWh(12),
                             to_MTperMWh(12),
                             to_MTperMWh(48),
                             to_MTperMWh(41),
                             to_MTperMWh(11),
                             0.0],
           'source':['\"NREL Annual Technology Baseline 2020\"',
                     '\"NREL Annual Technology Baseline 2020\"',
                     '\"NREL Annual Technology Baseline 2020\"',
                     '\"NREL Annual Technology Baseline 2020\"',
                     'IEA-2020',
                     '\"NREL Annual Technology Baseline 2020\"',
                     '\"NREL Annual Technology Baseline 2020\"',
                     '\"NREL Annual Technology Baseline 2020\"',
                     '\"NREL Annual Technology Baseline 2020\"',
                     '\"NREL Annual Technology Baseline 2020\", MISC'],
           'notes':['based on average capacity factor (54%)',
                    'with 90% CCS and average cap. factor.',
                    'combined cycle with average capacity factor (55%)',
                    'CC with carbon capture (90%)',
                    'current LWR, investment cost refers to cost of license renewal',
                    'advanced or modular',
                    'utility scale solar, photovoltaic, chicago', 
                    'residential solar, photovoltaic, chicago', 
                    'utility scale wind turbines, assume Class 7 wind speeds',
                    'lithium ion battery'],
           'notes-co2eq':['co2eq is the median value from the Fifth IPCC Assessement',
                          'co2eq is the median value from the Fifth IPCC Assessement',
                          'co2eq is the median value from the Fifth IPCC Assessement',
                          'co2eq is the median value from the Fifth IPCC Assessement',
                          'co2eq is the median value from the Fifth IPCC Assessement',
                          'co2eq is the median value from the Fifth IPCC Assessement', 
                          'co2eq is the median value from the Fifth IPCC Assessement',
                          'co2eq is the median value from the Fifth IPCC Assessement',
                          'co2eq is the median value from the Fifth IPCC Assessement',
                          'n/a']}

In [36]:
cost_df = pd.DataFrame(cost_df)
cost_df

,technology,investment (M$/MW),fixed (M$/MW-year),variable (M$/MWh),SO2 (MT/MWh),NOx (MT/MWh),CO2 (MT/MWh),Hg (MT/MWh),CO2eq (MT/MWh),source,notes,notes-co2eq
0,coal_existing,3.666347,0.040703,0.000021,1.547720e-10,1.238176e-10,3.259499e-07,6.749608e-15,8.200000e-07,"""NREL Annual Technology Baseline 2020""",based on average capacity factor (54%),co2eq is the median value from the Fifth IPCC ...
1,coal_new,6.035277,0.059020,0.000037,8.589847e-11,1.315562e-10,3.265690e-08,6.749608e-16,2.200000e-07,"""NREL Annual Technology Baseline 2020""",with 90% CCS and average cap. factor.,co2eq is the median value from the Fifth IPCC ...
2,natgas_existing,0.959580,0.011193,0.000022,5.107476e-12,3.095440e-11,1.810833e-07,0.000000e+00,4.900000e-07,"""NREL Annual Technology Baseline 2020""",combined cycle with average capacity factor (55%),co2eq is the median value from the Fifth IPCC ...
3,natgas_new,2.712873,0.027475,0.000027,5.107476e-12,3.095440e-11,1.810833e-08,0.000000e+00,1.700000e-07,"""NREL Annual Technology Baseline 2020""",CC with carbon capture (90%),co2eq is the median value from the Fifth IPCC ...
4,nuclear_existing,0.391000,0.057413,0.000030,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.200000e-08,IEA-2020,"current LWR, investment cost refers to cost of...",co2eq is the median value from the Fifth IPCC ...
5,nuclear_new,6.232688,0.121092,0.000009,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.200000e-08,"""NREL Annual Technology Baseline 2020""",advanced or modular,co2eq is the median value from the Fifth IPCC ...
6,solar_utility,1.593533,0.019334,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.800000e-08,"""NREL Annual Technology Baseline 2020""","utility scale solar, photovoltaic, chicago",co2eq is the median value from the Fifth IPCC ...
7,solar_residential,3.107694,0.022387,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.100000e-08,"""NREL Annual Technology Baseline 2020""","residential solar, photovoltaic, chicago",co2eq is the median value from the Fifth IPCC ...
8,wind_utility,1.878456,0.043756,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.100000e-08,"""NREL Annual Technology Baseline 2020""","utility scale wind turbines, assume Class 7 wi...",co2eq is the median value from the Fifth IPCC ...
9,li_battery,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,"""NREL Annual Technology Baseline 2020"", MISC",lithium ion battery,n/a


In [37]:
cost_df.to_csv("cost_emissions_data.csv")

## Battery Storage

In [38]:
om_battery50_200_fixed_sl = inflate(24.8*1000/1e6, 2020)
om_battery50_100_fixed_sl = inflate(12.9*1000/1e6, 2020)

battery_df = {'capacity (MW)':[50.0,
                               50.0,],
              'duration (MWh)':[100.0,
                                200.0,],
              'capital (M$/MW)':[inflate(845*1000/1e6,2020),
                                 inflate(1389*1000/1e6,2020),],
              'fixed (M$/MW-year)':[om_battery50_100_fixed_sl,
                                    om_battery50_200_fixed_sl,
                                   ],
              'source':['Sargent&Lundy 2020',
                        'Sargent&Lundy 2020'],
              'notes':['',
                       '',]}

In [39]:
battery_df = pd.DataFrame(battery_df)
battery_df

,capacity (MW),duration (MWh),capital (M$/MW),fixed (M$/MW-year),source,notes
0,50.0,100.0,0.859857,0.013127,Sargent&Lundy 2020,
1,50.0,200.0,1.413421,0.025236,Sargent&Lundy 2020,


## Nuclear Fixed Cost: Incorporating Guaranteed Generation


Nuclear plants do not "load follow" the way natural gas plants do because of the 
physical laws governing fission in the reactor's core. Thus, nuclear plants have a 
high capacity factor, around 92.5\% according to EIA and the DOE. 

In order for Temoa to properly use existing nuclear resources, we should calculate ahead of time the total electricity generated by Illinois' nuclear fleet and incorporate this into the fixed cost of nuclear.

$$ FC_{adj} = CF*Hours/Year*P $$

#### What should the fixed price be?

There are actually two prices: 
1. The breakeven cost of producing electricity for a nuclear plant.
2. The price the nuclear plant actually gets paid. 

BAU shows how much nuclear generation the market _values_. BAU2 shows how much nuclear generation the market _gets_ and the difference is the deficit for nuclear plants.

SD4 is the scenario where nuclear is appropriately valued.

In [40]:
fc_adjusted_be = om_nei_bwr_total*8760*0.925*1000
print(f"The fixed cost adjusted for nuclear is {fc_adjusted_be}")

fc_adjusted_be = natgas_existing_variable*8760*0.925*1000
print(f"The fixed cost adjusted for nuclear (at natural gas price) is {fc_adjusted_be}")

The fixed cost adjusted for nuclear is 242.12974514034076
The fixed cost adjusted for nuclear (at natural gas price) is 181.4002177114988


In [41]:
p = inflate(31.52, 2019)

In [42]:
p*8760*0.925/1e6*1000  # M$/MWh * 1000

263.1415258710145

In [43]:
pc = inflate(5.83/1e6*1000, 2019)

In [44]:
pc

0.006006561052417816

In [45]:
natgas_existing_variable*1000

0.02238679719998751

In [46]:
natgas_new_variable*1000

0.027474705654530123